<a href="https://colab.research.google.com/github/thejoshchow/fraud-detection/blob/ops/src/Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dataaspirant.com/build-email-spam-classification-model-spacy-python/

In [ ]:
import pandas as pd

In [ ]:
import random

The json read was not used. Instead see below the read of text2.csv

In [ ]:
fraud=pd.read_csv('/content/data/data.json')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
parsed=pd.read_csv('/content/data/text2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


drop nan rows

In [ ]:
checks_all=([isinstance(value,str) for value in   parsed['description'].values])

In [ ]:
parsed_checked=parsed[checks_all]

The label column becomes y

In [ ]:
X=parsed_checked.iloc[:,1:4]
X.head(5)
y=parsed_checked.iloc[:,5]
y.head(5)

0     True
1    False
2    False
3    False
4    False
Name: fraud, dtype: object

In [ ]:
import seaborn as sns
from spacy.util import minibatch
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt 

In [ ]:
import spacy as spacy

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
nlp=spacy.blank('en')

In [ ]:
text_cat=nlp.create_pipe('textcat',config=({'exclusive_classes':True,'architecture':'bow'}))

In [ ]:
nlp.add_pipe(text_cat)

In [ ]:
text_cat.add_label('not_fraud')
text_cat.add_label('fraud')

1

In [ ]:
text_cat

Split inputs intro training and test groups

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=20)

In [ ]:
y_train[0:5]

10582    False
1953     False
8823     False
4302     False
10081    False
Name: fraud, dtype: object

In [ ]:
train_labels=[{'cats':{'fraud':label=='True','not_fraud':label=='False'}} for label in y_train]
test_labels=[{'cats':{'fraud':label=='True','not_fraud':label=='False'}} for label in y_test]

In [ ]:
train_labels

[{'cats': {'fraud': True, 'not_fraud': False}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': False}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': True, 'not_fraud': False}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud': True}},
 {'cats': {'fraud': False, 'not_fraud':

In [ ]:
train_data=list(zip(((X_train['name'].values)[0:99]),(train_labels[0:99])))
test_data=list((zip((X_test['name'].values)[0:99]),(test_labels[0:99])))

In [ ]:
def train_model(model,train_data,optimizer, batch_size,epochs=10): 
  losses={}
  random.seed(1)

  for epoch in range(epochs):
    random.shuffle(train_data)

    batches=minibatch(train_data,size=batch_size)
    for batch in batches:
      texts,labels=zip(*batch)

      model.update(texts,labels,sgd=optimizer,losses=losses)
    print('Loss:{}'.format(losses['textcat']))
  return losses['textcat']


Training using the name column

In [ ]:

optimizer=nlp.begin_training()
batch_size=5

epochs=10

train_model(nlp,train_data,optimizer,batch_size,epochs)

Loss:0.08963124768342823
Loss:0.16832079022424296
Loss:0.24127827800111845
Loss:0.3102858355268836
Loss:0.3708487557596527
Loss:0.42933617962989956
Loss:0.48143584790523164
Loss:0.5293489998148289
Loss:0.5755694067847799
Loss:0.6183888778250548


0.6183888778250548

In [ ]:
print(train_data[0])

('Manifest Destiny Conference 2010', {'cats': {'fraud': True, 'not_fraud': False}})


In [ ]:
sample_test=nlp(train_data[0][0])
print(sample_test.cats)

{'fraud': 0.6707589626312256, 'not_fraud': 0.3292410373687744}


In [ ]:
def get_predictions(model,texts):
  docs=[model.tokenizer(text) for text in texts]

  textcat=model.get_pipe('textcat')
  scores,_=textcat.predict(docs)

  predicted_labels=scores.argmax(axis=1)
  predicted_class=[textcat.labels[labels] for labels in predicted_labels]
  
  return predicted_labels

In [ ]:
 docs=[text for text in X_train['name']]

In [ ]:
X_train

,name,description,org_name
463,LeDiCe GRADUATION PARTY,<p>#YOU MADE IT.. LETS PARTY FOR THE LAST TIME...,LeDiCe
2697,Shropshire Cadets Concert,"<p><span style=""color: #222222; font-family: a...",Siobhan Kilbane and Ray Wickson on Behalf of C...
10484,Wine Down Tour (Labor Day Weekend),"<p><span style=""font-size: small;"">This Labor ...",WineMeDown Events
10094,RENEGADETHEBEAST AND FRIENDS 2012 CALENDAR,<p>RENEGADETHEBEAST THE HOTTEST DANCER UNDER T...,atlstripper
3996,Bar club,<p>Bar club</p>,NaN
...,...,...,...
9620,Lead with Your Words: Biblical Principles for ...,"<p><img alt=""Bill Harris Poster"" width=""612"" ...",Wellspring School of Leadership
7068,BYT Presents: Get Beaujolaid @ 1905,"<p style=""text-align: left;""><font size=""5""><b...",BrightestYoungThings.com
7391,Essence Music Festival 2013,"<p><strong style=""color: #2c2d2f; font-family:...",Backside Entertainment Powered by The SETNETWORK
4367,BCAA Leadership Club -Communication with Emoti...,"<div style=""text-align: -webkit-auto;"">\n<div ...",BCAA


In [ ]:
train_predictions=get_predictions(nlp,(X_train['name'].values)[0:99])
test_predictions=get_predictions(nlp,(X_test['name'].values)[0:45])
train_accuracy=accuracy_score(y_train[0:99],train_predictions)
test_accuracy=accuracy_score(y_test[0:45],test_predictions)

In [ ]:
print(train_accuracy,test_accuracy)

0.0 0.0


In [ ]:
print(np.array(test_predictions))

['not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud' 'not_fraud'
 'not_fraud' 'not_fraud' 'not_fraud']


In [ ]:
print(y_train[0:45])

463                               True
2697                             False
10484    wine tasting experiences.</p>
10094                            False
3996                              True
5991                             False
2047                             False
13774                            False
9847                             False
12996                            False
482                              False
10471                            False
12807                            False
3122                             False
9860                             False
4907                             False
2274                             False
3722                             False
3978                             False
819                              False
7823                             False
12763                            False
7757                             False
13426                            False
7607                             False
9255                     

In [ ]:
train_data=list(zip(((X_train['description'].values)[25:100]),(train_labels[25:100])))
test_data=list((zip((X_test['description'].values)[25:50]),(test_labels[25:50])))

Training using the description column

In [ ]:
train_data_d=list(zip(((X_train['description'].values)),(train_labels)))
test_data_d=list((zip((X_test['description'].values)),(test_labels)))

In [ ]:

optimizer_d=nlp.begin_training()
batch_size=5

epochs=10

train_model(nlp,train_data_d,optimizer_d,batch_size,epochs)

Loss:11.02251581125786
Loss:17.73043722358214
Loss:21.64153189152224
Loss:24.323790010662123
Loss:26.549963389481878
Loss:28.399137657845085
Loss:30.11444793140428
Loss:31.6119076214801
Loss:32.92711531314343
Loss:34.23373766171704


34.23373766171704

In [ ]:
train_prediction_d=get_predictions(nlp,((X_train['description'].values)))
test_predictions_d=get_predictions(nlp,(X_test['description'].values))
#train_accuracy=accuracy_score(y_train,train_predictions)
#test_accuracy=accuracy_score(y_test,test_predictions==')

In [ ]:
print(list(test_predictions_d==(y_test.values=='True'))[0:50])

[True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [ ]:
sum(list(test_predictions_d==(y_test.values=='True'))[0:50])

49

In [ ]:
train_accuracy=accuracy_score(y_train.values=='True',train_prediction_d)
test_accuracy=accuracy_score(y_test.values=='True',test_predictions_d)

In [76]:
from sklearn.metrics import precision_score,recall_score

In [75]:
precision_score(y_test.values=='True',test_predictions_d)

0.6492537313432836

In [77]:
recall_score(y_test.values=='True',test_predictions_d)

0.4264705882352941

In [ ]:
print('train_accuracy',train_accuracy,'test_accuracy',test_accuracy)

train_accuracy 0.9930600536689183 test_accuracy 0.9393042190969653
